**Using BERTweet-base-finetuned-combined-DS to label S3D**

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("surrey-nlp/bertweet-base-finetuned-SARC-DS", max_length = 128, truncation = True, padding='max_length')

model = AutoModelForSequenceClassification.from_pretrained("surrey-nlp/bertweet-base-finetuned-SARC-DS")

Downloading:   0%|          | 0.00/338 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/819 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/s3dpreprocessed/S3D-preprocessed.csv', lineterminator = '\n')
df = df['Text']
df = df.dropna(axis=0, how="any", inplace=False)

In [3]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, truncation=True, device= 0, binary_output=True)
data = list(df)
out = pipe(data)

/opt/conda/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


In [4]:
def get_preds(out):

    labels = []
    for i in range(len(out)):
        if out[i][0]['score'] > out[i][1]['score']:
            labels.append(0)
        else:
            labels.append(1)
            
    return labels

In [5]:
labels = get_preds(out)
df2 = pd.DataFrame(labels, columns = ['Label'])
df3 = pd.concat([df, df2], axis = 1)

In [6]:
df3[0:1000]

,Text,Label
0,@user new jersey not the big apple smh admin,1.0
1,the uk @user some 28bn of total wealth most of...,0.0
2,@user the feeling of a country being forced to...,0.0
3,@user @user @user i spend the entire summer by...,0.0
4,ill make a karma cometrueyeaahhh idgaf but at ...,1.0
...,...,...
995,terrorism upon @user while we beg for an @user...,1.0
996,@user @user lol no lets not the ccp has totall...,1.0
997,@user @user the story of how africans underdev...,1.0
998,she just give him double work and make him get...,1.0
